# MNIST-Convolutional Neural Netowrk
ถึงคราวพระเอกตัวจริงของเราออกโรง ในตอนนี้เราจะเปลี่ยนสถาปัตยกรรมมาเน้นที่ Convolutional Neural Network (CNN) และเราจะยกระดับโค้ดของเราให้มีการนิยามงานที่ทำซ้ำ ๆ เดิมเป็นฟังก์ชันด้วย จุดนี้จะทำให้เราสามารถยกโค้ดที่เราเรียนไปใช้ในงานจริงได้สะดวกกว่าเดิม

[เนื้อหาส่วนนี้ดัดแปลงมาจาก[เพจฝึกสอนของไมโครซอฟต์](https://github.com/Microsoft/CNTK/blob/master/Tutorials/CNTK_103D_MNIST_ConvolutionalNeuralNetwork.ipynb) แต่ได้ตัดเนื้อหาออกหลายส่วน สำหรับผู้ที่จำเป็นต้องใช้เทคนิคนี้อย่างจริงจังควรอ่านศึกษาข้อมูลในเพจฝึกสอนเพิ่มเติมเนื่องจากความเข้าใจในตัวสถาปัตยกรรมเป็นสิ่งที่จะเป็นในการปรับโครงสร้างในแม่แบบให้เหมาะกับงานที่เราต้องการประยุกต์ใช้ เราไม่สามารถเพิกเฉยต่อความเข้าใจเหล่านี้ได้]

### เริ่มต้นยังคล้ายเดิม ที่เพิ่มเติมคือ GPU
ตอนนี้เราจะใช้โค้ดสำหรับอ่านข้อมูลแบบเดิม แต่ตรงที่เกี่ยวกับโครงสร้างของโครงข่ายประสาทเทียมจะเปลี่ยนไปมาก เราจะพูดถึงรายละเอียดคร่าว ๆ อีกครั้งเมื่อไปถึงจุดนั้น อย่างไรก็ตาม ในครั้งนี้เราจำเป็นต้องบังคับใช้ GPU กันอย่างจริงจัง เพราะเราจะมีการทำคอนโวลูชันบนฟีทเจอร์แม็พจำนวนมาก หากไม่ใช้ GPU เราคงทนรอไม่ไหว

ดังนั้นอันกับแรกเราก็บังคับให้มันเรียกพลังของ GPU มาใช้งานเลย (ใครที่เครื่องมี GPU มากกว่า 1 ตัวอาจจะต้องเขียนโค้ดต่างจากนี้เล็กน้อย)

In [1]:
from cntk.device import set_default_device, gpu
set_default_device(gpu(0))

True

In [2]:
import numpy as np
import sys
import os
import math
import cntk

# Read a CTF formatted text (as mentioned above) using the CTF deserializer from a file
def create_reader(path, is_training, input_dim, num_label_classes):   
    labelStream = cntk.io.StreamDef(field='labels', shape=num_label_classes, is_sparse=False)
    featureStream = cntk.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    
    deserailizer = cntk.io.CTFDeserializer(path, cntk.io.StreamDefs(labels = labelStream, features = featureStream))
            
    return cntk.io.MinibatchSource(deserailizer,
       randomize = is_training, max_sweeps = cntk.io.INFINITELY_REPEAT if is_training else 1)

# Ensure the training and test data is generated and available for this tutorial.
# We search in two locations in the toolkit for the cached MNIST data set.
data_found = False

for data_dir in ["."]:
    train_file = os.path.join(data_dir, "Train-28x28_cntk_text.txt")
    test_file = os.path.join(data_dir, "Test-28x28_cntk_text.txt")
    if os.path.isfile(train_file) and os.path.isfile(test_file):
        data_found = True
        break
        
if not data_found:
    raise ValueError("Your data files are not available. Please check it out if you put them in the same fol")
    
print("Data directory is {0}".format(data_dir))
print("Train-data path is " + train_file)
print("Test-data path is " + test_file)

Data directory is .
Train-data path is .\Train-28x28_cntk_text.txt
Test-data path is .\Test-28x28_cntk_text.txt


### ต้องบรรยายข้อมูลเกี่ยวกับโครงสร้างสองมิติของข้อมูลด้วย
วิธีการที่ผ่านมา เรามองข้อมูลเข้าเป็นเวคเตอร์ยาว ๆ ไม่ได้มองว่ามันเป็นภาพสองมิติแต่อย่างใด ทำให้เราไม่สามารถผนวกเอาลักษณะการรับรู้ของมนุษย์เข้าไปนำทางการเรียนรู้ได้ อันดับแรกเราจะบอกลักษณะของภาพสองมิติในรูปแบบ (จำนวนสี, ความกว้าง, ความสูง) ซึ่งจำนวนสีของเราก็คือ 1 เพราะเป็นภาพเฉดเทา ส่วนความกว้างและความสูงมีค่าเป็น 28 พิกเซลด้วยกันทั้งคู่ ทำให้เราได้โค้ดสำหรับบอกลักษณะของภาพสองมิติที่เป็นอินพุตดังนี้

In [3]:
input_dim_model = (1, 28, 28)

ส่วน input_dim ก็ยังเป็น 784 เหมือนเดิม แต่ความนิยมในการเขียนโค้ดจะเปลี่ยนจากการเขียนว่า 784 ตรง ๆ ไปเป็น 28 * 28 แทน เราสามารถใช้แบบใดก็ได้ แต่ในที่นี้ขอเปลี่ยนตามความนิยม

In [4]:
input_dim = 28 * 28 # ยังบอกเป็นขนาดเต็ม เพื่อที่ตอนอ่านข้อมูลจากไฟล์จะได้อ่านมาครบจำนวน
num_output_classes = 10

เรายังเก็บตัวแปร input_dim ไว้ใช้ในการอ่านไฟล์ แต่ตอนนิยามชั้นอินพุต เราจะใช้ input_dim_model แทน เพราะการจัดโครงสร้างข้อมูลไม่ได้เป็นแบบเวคเตอร์ยาว ๆ อันเดียวอีกต่อไปแล้ว

In [5]:
input = cntk.input_variable(input_dim_model)  # สังเกตว่าเราใช้ input_dim_model เป็นพารามิเตอร์แทนการใช้ input_dim
label = cntk.input_variable(num_output_classes)

## เข้าใจแนวคิดของ CNN
แนวคิดหลักของ CNN ในงานนี้จะเป็นการนำตัวกรองสองมิติ (2D filter) มาทาบใส่พื้นที่ในภาพเพื่อตรวจดูว่าพื้นที่ที่ทาบอยู่นั้นมีคุณสมบัติที่ตรงกับสิ่งที่น่าจะเป็นลักษณะเฉพาะของวัตถุที่ต้องการจำแนกหรือไม่ ซึ่งในที่นี้วัตถุที่ต้องการจำแนกก็คือเลจ 0 ถึง 9

กระบวนการทาบนั้นเหมือนกับการทำ spatial convolution (คอนโวลูชันเชิงพื้นที่) ของตัวกรองภาพที่เราเรียนมาก่อนหน้า (พวก Mean filter หรือ Gaussian filter) ซึ่งถ้าใครยังนึกไม่ออก ก็ดูภาพประกอบทางด้านใต้ได้เลย (ต้องการการเชื่อมต่ออินเตอร์เน็ตเพื่อดึงภาพขึ้นมา)

ปัญหาคือลักษณะเฉพาะที่ว่านั้นคืออะไร และเราจะบอกเครื่องให้ทราบถึงลักษณะเฉพาะนี้ได้อย่างไร ในประเด็นนี้คำตอบก็คือ "เราไม่จำเป็นต้องรู้ว่าลักษณะเฉพาะคืออะไร เราจะให้เครื่องไปหาเอง" เพียงแต่เราต้องเตรียมทรัพยากรด้านการคำนวณให้เพียงพอที่เครื่องจะหาลักษณะเฉพาะเหล่านั้นได้

In [6]:
from IPython.display import display, Image

# ภาพประกอบจาก Microsoft CNTK Tutorial
Image(url="https://www.cntk.ai/jup/cntk103d_conv2d_final.gif", width= 300)

### Convolution กับ Stride *
จากภาพด้านบนจะเห็นได้ว่า เครื่องจะทาบตัวกรองขนาด 3x3 ไปเรื่อย ๆ ในภาพอินพุต ภาพอินพุตในตัวอย่างนั้นมีสามชั้นจากสีสามสีคือ Red, Green, Blue แต่ในงาน OCR ของเรา จะมีเพียงสีเดียว ซึ่งขอให้เข้าใจตรงกันว่าเทคนิค CNN ใช้กับภาพสีได้โดยสะดวก เราไม่ควรไปคิดว่าเราต้องแปลงภาพให้เป็นภาพเฉดเทาก่อน

เอาล่ะ **มาถึงงานของคุณแล้ว** คุณจะต้องอธิบายให้ได้ว่า Stride คืออะไรในบริบทนี้ และมีค่าเท่าใด พร้อมให้เหตุผลสนับสนุนคำตอบของคุณมาด้วย

### เราควรกำหนดค่า Stride อย่างไรถึงจะดี *
จงอธิบายว่าค่า stride น่าจะส่งผลกับการเรียนรู้ข้องเครื่องอย่างไรบ้าง และเราควรจะเลือกค่า Stride อย่างไรจึงจะได้ผลอย่างที่เราต้องการมากที่สุด (คำตอบของข้อนี้สัมพันธ์กับคำถามข้อต่อมา บางทีถ้าตอนแรกคิดข้อนี้ไม่ออก เราอาจจะไปคิดข้อต่อไปให้ออกก่อน แล้วมันอาจจะทำให้คิดข้อนี้ออกก็เป็นได้)

### Convolution กับจำนวนตัวกรอง และ Feature Maps *
จงอธิบายว่าตัวกรองกับ feature map มีความสัมพันธ์กันอย่างไรบ้างในเชิงของฟีทเจอร์ และการเพิ่มจำนวนตัวกรองจะทำให้ feature map เปลี่ยนแปลงอย่างไร

นอกจากนี้คุณคิดว่าเราจะกำหนดจำนวนตัวกรอง โดยใช้อะไรเป็ณเกณฑ์ในการพิจารณาบ้างถึงจะดี ในทางปฏิบัติเราควรจะศึกษาความสัมพันธ์ระหว่างจำนวนตัวกรองกับเกณฑ์เหล่านั้นอย่างไรจึงจะสามารถคาดการณ์ได้อย่างเป็นระบบตามกระบวนการทางวิทยาศาสตร์

### สร้างตัวแบบกันเลยดีกว่า
โค้ดข้างล่างนี้ นำมาจากเว็บฝึกสอนของไมโครซอฟต์ แต่เราจะใช้งานมันตรง ๆ ไม่ได้ เพราะเราเขียนโค้ดต่างจากทางไมโครซอฟต์นิดหน่อย

จงแก้โค้ดข้างลางนี้ เพื่อให้ฟังก์ชันสร้างตัวแบบ create_model สามารถทำงานได้ตามปรกติ

In [11]:
def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):
            h = features
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=8, 
                                       strides=(2,2), 
                                       pad=True, name='first_conv')(h)
            h = C.layers.Convolution2D(filter_shape=(5,5), 
                                       num_filters=16, 
                                       strides=(2,2), 
                                       pad=True, name='second_conv')(h)
            r = C.layers.Dense(num_output_classes, activation=None, name='classify')(h)
            return r

In [19]:
# Create the model
z = create_model(input)

# Print the output shapes / parameters of different components
print("Output Shape of the first convolution layer:", z.first_conv.shape)
print("Bias value of the last dense layer:", z.classify.b.value)

Output Shape of the first convolution layer: (32, 28, 28)
Bias value of the last dense layer: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]


In [20]:
# Number of parameters in the network
cntk.logging.log_number_of_parameters(z)

Training 98768 parameters in 9 parameter tensors.


In [21]:
def create_criterion_function(model, labels):
    loss = cntk.cross_entropy_with_softmax(model, labels)
    errs = cntk.classification_error(model, labels)
    return loss, errs # (model, labels) -> (loss, error metric)

In [22]:
# Define a utility function to compute the moving average sum.
# A more efficient implementation is possible with np.cumsum() function
def moving_average(a, w=5):
    if len(a) < w:
        return a[:]    # Need to send a copy of the array
    return [val if idx < w else sum(a[(idx-w):idx])/w for idx, val in enumerate(a)]


# Defines a utility that prints the training progress
def print_training_progress(trainer, mb, frequency, verbose=1):
    training_loss = "NA"
    eval_error = "NA"

    if mb%frequency == 0:
        training_loss = trainer.previous_minibatch_loss_average
        eval_error = trainer.previous_minibatch_evaluation_average
        if verbose: 
            print ("Minibatch: {0}, Loss: {1:.4f}, Error: {2:.2f}%".format(mb, training_loss, eval_error*100))
        
    return mb, training_loss, eval_error

In [23]:
import time

def train_test(train_reader, test_reader, model_func, num_sweeps_to_train_with=10):
    
    # Instantiate the model function; x is the input (feature) variable 
    # We will scale the input image pixels within 0-1 range by dividing all input value by 255.
    model = model_func(input/255)
    
    # Instantiate the loss and error function
    loss, label_error = create_criterion_function(model, label)
    
    # Instantiate the trainer object to drive the model training
    learning_rate = 0.2
    lr_schedule = cntk.learning_rate_schedule(learning_rate, cntk.UnitType.minibatch)
    learner = cntk.sgd(z.parameters, lr_schedule)
    trainer = cntk.Trainer(z, (loss, label_error), [learner])
    
    # Initialize the parameters for the trainer
    minibatch_size = 64
    num_samples_per_sweep = 60000
    num_minibatches_to_train = (num_samples_per_sweep * num_sweeps_to_train_with) / minibatch_size
    
    # Map the data streams to the input and labels.
    input_map={
        label  : train_reader.streams.labels,
        input  : train_reader.streams.features
    } 
    
    # Uncomment below for more detailed logging
    training_progress_output_freq = 500
     
    # Start a timer
    start = time.time()

    for i in range(0, int(num_minibatches_to_train)):
        # Read a mini batch from the training data file
        data=train_reader.next_minibatch(minibatch_size, input_map=input_map) 
        trainer.train_minibatch(data)
        print_training_progress(trainer, i, training_progress_output_freq, verbose=1)
     
    # Print training time
    print("Training took {:.1f} sec".format(time.time() - start))
    
    # Test the model
    test_input_map = {
        label  : test_reader.streams.labels,
        input  : test_reader.streams.features
    }

    # Test data for trained model
    test_minibatch_size = 512
    num_samples = 10000
    num_minibatches_to_test = num_samples // test_minibatch_size

    test_result = 0.0   

    for i in range(num_minibatches_to_test):
    
        # We are loading test data in batches specified by test_minibatch_size
        # Each data point in the minibatch is a MNIST digit image of 784 dimensions 
        # with one pixel per dimension that we will encode / decode with the 
        # trained model.
        data = test_reader.next_minibatch(test_minibatch_size, input_map=test_input_map)
        eval_error = trainer.test_minibatch(data)
        test_result = test_result + eval_error

    # Average of evaluation errors of all test minibatches
    print("Average test error: {0:.2f}%".format(test_result*100 / num_minibatches_to_test))

## ฝึกตัวแบบและทำการทดสอบความแม่นยำ
มาถึงส่วนที่ใช้เวลานานสักหน่อย ซึ่งก็คือการฝึกตัวแบบ โดยเมื่อฝึกเสร็จแล้ว เราจะทำการทดสอบตามมาทันที และ Average test error จะเป็นหนึ่งในตัวชี้ที่สำคัญว่าตัวแบบที่เราสร้างขึ้นมา จริง ๆ แล้วมันดีจริงหรือเปล่า

In [24]:
def do_train_test():
    global z
    z = create_model(input)
    reader_train = create_reader(train_file, True, input_dim, num_output_classes)
    reader_test = create_reader(test_file, False, input_dim, num_output_classes)
    train_test(reader_train, reader_test, z)
    
do_train_test()

Minibatch: 0, Loss: 2.3085, Error: 87.50%
Minibatch: 500, Loss: 0.1289, Error: 4.69%
Minibatch: 1000, Loss: 0.0911, Error: 3.12%
Minibatch: 1500, Loss: 0.1675, Error: 1.56%
Minibatch: 2000, Loss: 0.0078, Error: 0.00%
Minibatch: 2500, Loss: 0.0028, Error: 0.00%
Minibatch: 3000, Loss: 0.0427, Error: 1.56%
Minibatch: 3500, Loss: 0.0354, Error: 0.00%
Minibatch: 4000, Loss: 0.0039, Error: 0.00%
Minibatch: 4500, Loss: 0.0240, Error: 1.56%
Minibatch: 5000, Loss: 0.0181, Error: 1.56%
Minibatch: 5500, Loss: 0.0011, Error: 0.00%
Minibatch: 6000, Loss: 0.0130, Error: 0.00%
Minibatch: 6500, Loss: 0.0120, Error: 0.00%
Minibatch: 7000, Loss: 0.0346, Error: 1.56%
Minibatch: 7500, Loss: 0.0008, Error: 0.00%
Minibatch: 8000, Loss: 0.0032, Error: 0.00%
Minibatch: 8500, Loss: 0.0169, Error: 0.00%
Minibatch: 9000, Loss: 0.0006, Error: 0.00%
Training took 44.0 sec
Average test error: 0.86%


### ตกลงตัวแบบที่ได้มาดีพอหรือยัง
เราได้ค่าความผิดพลาดอยู่ที่ประมาณ 1.4 - 1.6% ซึ่งก็ถือว่าดีกว่าที่ผ่านมาทั้งหมด แต่ที่จริงการจะทำให้ความผิดพลาดเหลือเพียง 0.8% หรือดีกว่านับว่าเป็นเรื่องธรรมดา แต่เราจะต้องสร้างตัวแบบให้ถูกสุขลักษณะกว่านี้ อย่างแรกก็คือ เรายังไม่มีการทำ Pooling อย่างที่สองก็คือ เรายังไม่มีการทำ dropout และสุดท้าย เราอาจจะใช้ฟีทเจอร์แมพน้อยไป ไม่ได้ประสานกับการทำ Pooling หรือใช้ stride ในการทำคอนโวลูชันใหญ่เกินไปก็เป็นไปได้

แต่ก่อนที่จะไปทำการแก้ไขฟังก์ชัน เรามาทบทวนกันก่อนดีกว่า ว่าเราเข้าใจแนวคิดของงานแต่ละอย่างถูกต้องหรือยัง

### จงอธิบายว่าแนวคิดของการทำคอนโวลูชันดีกว่าวิธีการเชื่อมต่อกันหมดได้อย่างไรในแง่ของความแม่นยำของตัวแบบ *

จะใช้รูปประกอบในการตอบคำถามก็ได้

### จงอธิบายแนวคิดของการทำ Pooling ว่ามีประโยชน์อย่างไร *

พร้อมอธิบายด้วยว่า ทำไมเราถึงทำ Max Pooling ไม่ทำ Min Pooling

### จงอธิบายประโยชน์ของ dropout ว่ามันใช้แก้ปัญหาอะไร และแก้ได้อย่างไร *


### โอเคงั้นมาสร้างโมเดลใหม่ เอาแบบจัดเต็มให้ดีกว่าเดิม

คราวนี้เราจะเขียนฟังก์ชัน create_model ใหม่ในเซลล์ข้างล่างนี้ จากนั้นเราจะสั่งรันเซลล์ข้างล่าง แล้ววกกลับรันเซลล์ที่ขึ้นต้นด้วย
z = create_model(input)

จากนั้นเราจะรันเซลล์ต่าง ๆ ที่ตามมาและดูว่าความผิดพลาดเหลือเท่าใด อ่อ เวลาที่ต้องใช้ในการฝึกมันจะเพิ่มขึ้นมาก เตรียมใจไว้หน่อย แต่ผลลัพธ์มันจะดีขึ้นมาก ความผิดพลาดลดลงไปอย่างมีนัยสำคัญเมื่อเทียบเป็นร้อยละของพัฒนาการ

สำหรับตัวแบบของเรานั้นจะมีสถาปัตยกรรมดังนี้
ต่อจากชั้นอินพุต เราจะเตรียมชั้นต่าง ๆ ตามลำดับต่อไปนี้
ชั้นแรก: ทำคอนโวลูชัน ขนาด 5x5, ฟิลเตอร์ 32 ตัว, stride = (1, 1) และมีการ pad
ชั้นที่สอง: ทำ max pooling ขนาด 3x3 และมี stride = (2, 2)
ชั้นที่สาม: ทำคอนโวลูชัน ชนาด 3x3, ฟิลเตอร์ 48 ตัว, stride = (1, 1) ไม่มีการ pad
ชั้นที่สี่: ทำ max pooling แบบเดียวกับที่ทำในชั้นที่สอง
ชั้นที่ห้า: ทำคอนโวลูชันเหมือนชันที่สาม แต่ให้เพิ่มฟิลเตอร์เป็น 64 ตัว
ชั้นที่หก: ทำชั้นแบบ Dense ที่มีจำนวนโหนด 96 โหนด
ชั้นที่เจ็ด: ทำ Dropout โดยมี dropout_rate ที่ 50% (ชื่อเลเยอร์ในโค้ดคือ cntk.layers.Dropout ลองค้นเอกสารหาความรู้เพิ่มเติมดู)
ชั้นที่แปด: ชั้นผลลัพธ์ ในกรณีนี้ เป็นการต่อจากชั้น Dropout ดังนั้นเราจะใช้ชั้นแบบ Embedding ที่มีจำนวนโหนดเท่ากันจำนวนคลาส เพื่อให้มันคำนวณผลลัพธ์ออกมาให้เรา

**หมายเหตุ** เมื่อเรารันเซลล์ข้างล่างนี้ ระบบจะนิยามฟังก์ชัน create_model ทับอันเดิม ทำให้เราเรียกใช้โค้ดสร้างโมเดลแบบเดิมได้ แต่จะได้โมเดลใหม่ไปแทน

In [ ]:
def create_model(features):
    with ????.layers.default_options(init=????.glorot_uniform(), activation=????.relu):
        
        return r

-- จบบริบูรณ์